Este notebook foi feito no google colab. Algumas funcionalidades podem ser diferentes das de outros editores como o jupyter notebook, por exemplo.


In [ ]:
#Instalar o pyradiomics no google colab

!pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 38.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 36.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 10.8 MB/s eta 0:00:

In [ ]:
#Importar bibliotecas necessárias

import os
from pathlib import Path
import numpy as np
import pandas as pd
from radiomics import featureextractor
import SimpleITK as sitk
import six

In [ ]:
#Importar o google drive para o notebook

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Diretório com as imagens

images_dir = Path(r"/content/drive/MyDrive/lidc/images_and_masks")

In [ ]:
#Lista com os pacientes

all_patients = sorted(list(images_dir.glob("LID*")))
all_patients[0:5]

[PosixPath('/content/drive/MyDrive/lidc/images_and_masks/LIDC-IDRI-0001'),
 PosixPath('/content/drive/MyDrive/lidc/images_and_masks/LIDC-IDRI-0002'),
 PosixPath('/content/drive/MyDrive/lidc/images_and_masks/LIDC-IDRI-0003'),
 PosixPath('/content/drive/MyDrive/lidc/images_and_masks/LIDC-IDRI-0004'),
 PosixPath('/content/drive/MyDrive/lidc/images_and_masks/LIDC-IDRI-0005')]

In [ ]:
#Função que retorna o número que apareceu mais vezes numa lista e quantas vezes apareceu

def most_frequent_number(lst):

    unique_values, counts = np.unique(lst, return_counts=True)
    max_count_index = np.argmax(counts)
    most_frequent = unique_values[max_count_index]

    return most_frequent, counts[max_count_index]

In [ ]:
"""
Função recursiva para reduzir o número de anotações de um nódulo
Tira um valor da lista até que o desvio padrão seja menor que 0.5 com um máximo de duas iterações
Após duas iterações, se o desvio padrão for maior que 0.5 retorna None
Se a lista tiver 3 valores e forem todos diferentes retorna None
"""

def reduce(values, iterations=0):
    std = np.std(values)

    if std <= 0.5:
        return values

    if len(values) == 3:
        result, times = most_frequent_number(values)
        if times != 2: #Se não houver um número que apareça pelo menos 2 vezes retorna None
            return None

    if iterations == 2:
        return None

    values_with_out_min = values[1:]
    values_with_out_max = values[:-1]

    if np.std(values_with_out_min) <= np.std(values_with_out_max):
        return reduce(values_with_out_min, iterations+1)
    else:
        return reduce(values_with_out_max, iterations+1)


Achamos que deveriamos discriminar os nódulos através do desvio padrão das suas anotações por considerarmos que quando o desvio padrão é menor que 0.5 os médicos estiveram perto de encontrar um consenso.

In [ ]:
#Extrair as features das imagens e respetivas segmentações

extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.addProvenance(provenance_on=False)

features_dataframe = pd.DataFrame() #Dataframe final com as features

for patient in all_patients:
    nodules = sorted(list(patient.glob("*"))) #Lista com os nódulos do paciente

    for nodule in nodules:
    temp_nodule_dataframe = pd.DataFrame() #Dataframe temporário para o nódulo
    nodule_dataframe = pd.DataFrame() #Dataframe final para o nódulo

    annotations = sorted(list(nodule.glob("*"))) #Lista com as anotações do nódulo

    malignancies = {} #Dicionário com as malignancies

    for annotation in annotations: #Loop para descobrir a malignancy final
        number = str(annotation)[-11:] #Número da anotação

        with open(f'{annotation}/malignancy.txt', 'r') as file:
            malignancy = file.read()
        malignancies[number] = int(malignancy) #Adiciona a malignancy ao dicionário

        values_list = sorted(list(malignancies.values())) #Lista com os valores das malignancies
        reduced_list = reduce(values_list) #Lista com os valores reduzida

    if reduced_list is not None:
        mean_malignancy = np.mean(reduced_list) #Malignancy média

        if mean_malignancy % 1 == 0.5: #Se acabar em .5, 2.5 por exemplo

            #A malignancy final do nódulo é a malignancy média arredondada pela função round() + 1,
            #Tem que ser a média mais 1 porque a função round() arredonda os números que terminam em .5 por defeito
            final_malignancy = round(mean_malignancy)+1 


            final_malignancies = [final_malignancy-1, final_malignancy] #Lista com os valores das malignancies que vão
                                                                        #contar para ser extraidas as features, no caso
                                                                        #da média ter dado 2.5 vão ser extraidas as features
                                                                        #das anotações que tiverem malignancy de 2 e de 3,
                                                                        #apesar da malignancy final ser 3
        else: #Se a malignancy não acabar em 0.5
            final_malignancy = round(mean_malignancy) #A malignancy final do nódulo é a malignancy média arredondada
            final_malignancies = [final_malignancy] #Neste caso extrai as features só dos nódulos com a malignancy final

        for annotation in annotations: #Loop para extrair as features

            with open(f'{annotation}/malignancy.txt', 'r') as file:
                annotation_malignancy = int(file.read())

            if annotation_malignancy in final_malignancies: #Se a malignancy da anotações for uma das escolhidas para extrair as features
                image_path = f'{annotation}/image.nrrd'
                mask_path = f'{annotation}/mask.nrrd'
                results = extractor.execute(image_path, mask_path) #Extrair as features

                features = []
                values = []
                for key, value in six.iteritems(results): #Criar lista com o nome das features e outra com os valores de cada uma delas
                    features.append(key)
                    values.append(value)

                annotation_dataframe = pd.DataFrame([values], columns=features) #Dataframe para a anotação

                #Concatena o dataframe da anotação e o temporário do nódulo
                temp_nodule_dataframe = pd.concat([nodule_dataframe, annotation_dataframe], axis=0) 

        column_names = temp_nodule_dataframe.columns.tolist() #Lista com os nomes das features

        nodule_values = [] #Lista com os valores do dataset
        nodule_columns = [] #Lista com o nome das colunas

        nodule_id = str(nodule)[45:]
        nodule_values.append(str(nodule)[45:]) #Adiciona o ID
        nodule_columns.append("ID")

        for column in column_names:

            mean_value = temp_nodule_dataframe[column].mean() #Calcula a média de cada features
            nodule_values.append(mean_value) #Adiciona o valor à respetiva lista
            nodule_columns.append(column) #Adiciona a feature à respetiva lista

        nodule_values.append(final_malignancy) #Adiciona a malignancy
        nodule_columns.append("malignancy")


        nodule_dataframe = pd.DataFrame([nodule_values], columns=nodule_columns) #Cria o datafram do nódulo

        features_dataframe = pd.concat([features_dataframe, nodule_dataframe], axis=0, ignore_index=True) #Concatena o final das features com o do nódulo

    features_dataframe.to_csv('/content/drive/MyDrive/lidc/features.csv', index=False) #Guarda o dataframe num ficheiro csv


A saída de streaming foi truncada nas últimas 5000 linhas.
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Co